# Retrieval Augmented Generation (RAG)
We basically first retrieve data using a search engine trained to our data, and then generate an answer based on that via LLMs

In [33]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-13 18:07:28--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-13 18:07:28 (26.9 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [34]:
import minsearch # alexeys small and fast search engine
import json
from dotenv import load_dotenv
from openai import OpenAI

In [35]:
# load environment variables
load_dotenv()
client = OpenAI()

## Start and train search Engine

In [36]:
# load json data 
# clean json dictionaries extracted by alexey grigorev using:
# https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/parse-faq.ipynb
# raw input data was the FAQ google documents for the mlops, ml, data engineering zoomcamp

with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [37]:
# rearrange data a bit (add course type to each faq)
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course'] #adding it to every faq
        documents.append(doc)

In [38]:
# initialize class, tell the search engine what is searchable and what are keywords
index = minsearch.Index(
    text_fields=['text','section','question'],
    keyword_fields=['course']
)

#actually train the search engine
index.fit(docs=documents)

## Generate LLM

In [39]:
#aux fcns
def search(query,filter_dict={'course': 'data-engineering-zoomcamp'}):
    '''  
    This function runs the already trained search engine and retrieves the top 5 results,
    '''
    boost = {'question': 3.0, 'section': 0.5} # what to stress on, what is more important. give it weights
    results = index.search(
        query=query,
        filter_dict=filter_dict,
        boost_dict=boost,
        num_results=10
    )

    return results

In [40]:
def build_prompt(query, search_results):
    ''' 
    This function starts with a prompt template, and given the query fills the template out with the results from the search engine
    '''
    # we will give the llm some context
    # Alexey mentions that this is a bit of art and science because you somewhat iterate until you find something that works for you.
    prompt_template =  """ 

    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT does not contain the answer, output NONE.

    QUESTION: {question}

    CONTEXT: {context}

    """.strip() #no line breaks
    
    #convert search results into proper formatted context
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"


    # we formally add the info on the prompt
    return prompt_template.format(question=query,context=context).strip()
    

In [41]:
def llm(prompt,model='gpt-4o-mini'):
    ''' 
    This function trains chatGPT with our prompt (with the search engine results)
    '''
    # train openai/chatpgt with the prompt
    response = client.chat.completions.create(
        model = model,
        messages=[{'role':'user','content': prompt}]
    )   

    return response.choices[0].message.content

In [42]:
def rag(query):
    '''  
    This function, given a question, finds best answers on the search engine, trains the llm with it, and returns a result
    '''
    # search for the question on the search engine
    results = search(q)
    # we create the context by basically stringing together the answers from the search engine
    prompt = build_prompt(q, results)
    # we train the llm (in this case chatGPT) with the prompt and returns some user friendly answer
    answer = llm(prompt)

    return answer



In [43]:
q= 'The course just started. Can I still enroll?'
answer = rag(q)
print(answer)

Yes, you can still enroll in the course after the start date. Even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so it's advisable not to leave everything for the last minute.


## Elasticsearch
Data will persist, because you trained it before.
This is what people formally use, we will use that to replace the toy `minsearch` engine

In [44]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [45]:
# create the elastic search client
es_client = Elasticsearch('http://localhost:9200')

#### Training the elasticsearch engine

In [46]:
# now we set up the index on elasticsearch
# this is the equivalent on minsearch to:
# index = minsearch.Index(
#     text_fields=['text','section','question'],
#     keyword_fields=['course']
# )

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

if not es_client.indices.exists(index=index_name):
    es_client.indices.create(index=index_name, body=index_settings)
else:
    print(f"Index '{index_name}' already exists.")

Index 'course-questions' already exists.


In [47]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [48]:
# we now actually index the data
# this is similar to the following in min search
# index.fit(docs=documents)
# note that you have to index every entry one by one

for doc in tqdm(documents):
    es_client.index(index=index_name,document=doc)

100%|██████████| 948/948 [00:01<00:00, 573.69it/s]


#### Redoing RAG with the pretrained elasticsearch engine

In [49]:
# set up a search query
# this is the equivalent on minsearch to:
# boost = {'question': 3.0, 'section': 0.5} # what to stress on, what is more important. give it weights
#     results = index.search(
#         query=query,
#         filter_dict=filter_dict,
#         boost_dict=boost,
#         num_results=10
#     )

def elastic_search(query,n_results=5):
    '''  
    This function submits a query to the pretrained elastic search, and returns formatted results
    default search results is 5
    question is 3x important than text and section
    '''

    # create properly formatted search query that works with elasticsearch
    search_query = {
        "size": n_results, #no of search results
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # question is 3x more important than text and section
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp" # we filter by course name
                    }
                }
            }
        }
    }

    # search the pretrained client given the formatted query
    response = es_client.search(index=index_name,body=search_query)

    # return all the top results in a readable python list
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [50]:
def rag(query):
    '''  
    This function, given a question, finds best answers on the search engine, trains the llm with it, and returns a result
    '''
    # search for the question on the search engine
    results = elastic_search(q)
    # we create the context by basically stringing together the answers from the search engine
    prompt = build_prompt(q, results)
    # we train the llm (in this case chatGPT) with the prompt and returns some user friendly answer
    answer = llm(prompt)

    return answer



In [51]:
answer = rag(q)
print(answer)

Yes, you can still enroll in the course after the start date. Even if you don't register, you're eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects. It's advised not to leave everything for the last minute.
